In [1]:
import ROOT, rat
import os, sys, pickle
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline 
sys.path.append('/home/jp/projects/snoplus/python_modules')
import jp_mpl as jplot
import geo_studies, rat_misc
from scipy import ndimage
reload(rat_misc)
from scipy import signal
from matplotlib.colors import LogNorm
from detect_peaks import detect_peaks
import pickle, scipy
from copy import deepcopy

TClass::TClass:0: RuntimeWarning: no dictionary for class omtext is available


In [2]:
# Airplane mode
db = rat.RAT.DB.Get()
db.SetAirplaneModeStatus(True)
db.SetDefaultPlaneLockStatus(False)
print 'This is AIRPLANE MODE - be careful!'

This is AIRPLANE MODE - be careful!


TClass::TClass:0: RuntimeWarning: no dictionary for class RAT::DBTableKey is available


In [3]:
socdir = '/home/jp/projects/snoplus/laserball_calibration/all_runs'
peakdir = '/home/jp/projects/snoplus/laserball_calibration/peak_output'

In [4]:
infile_list = os.listdir(socdir)
for i, x in enumerate(infile_list): print i, x 


0 SOC_0000017377_reprocessed.root
1 SOC_0000017375_reprocessed.root
2 SOC_0000017378_reprocessed.root
3 SOC_0000100558.root
4 SOC_0000100559.root
5 SOC_0000100560.root
6 SOC_0000100554.root
7 SOC_0000017386_reprocessed.root
8 SOC_0000100556.root
9 SOC_0000017376_reprocessed.root
10 SOC_0000100555.root
11 SOC_0000017384_reprocessed.root


In [5]:
bin_width = 0.5
npmts = 9728

In [6]:
def getTimeDiff(time_array,
                bin_width=0.5,
                twindow = [110, 220],
                plot = False):
    ybins  = np.linspace(0, 300, (300/bin_width)+1)
    smooth_coeff = 1./(ybins[1]-ybins[0])
    
    # Finding the peak positions
    n, x = np.histogram(time_array, ybins)
    nsmooth = ndimage.filters.gaussian_filter1d(n, sigma=smooth_coeff)
    
    # Will use log(n) - zeros go away
    nsmooth[nsmooth<=0] = 1

    # Peak finding algorithm
    peaks = detect_peaks(nsmooth, 
                         mpd = 10*smooth_coeff,
                         edge='rising',show=False, mph=1.1)
    
    # Obtain the highest peak
    highest_peak = peaks[nsmooth[peaks].argmax()]
    # Obtain the latest peak
    latest_peak = peaks[ybins[peaks].argmax()]
    # Getting the time difference between highest ones
    tdiff_peak = ybins[latest_peak] - ybins[highest_peak]
    
    # Finding the peaks of the derivative
    dx = (ybins[1]-ybins[0])/2.
    dn = np.diff(np.log10(n))/dx
    ydn = ybins[1:-1]

    # Derivating, and smoothing over it
    dnsmooth = np.diff(np.log10(nsmooth))/dx  
    dnsmooth = ndimage.filters.gaussian_filter1d(dnsmooth, 
                                                 sigma=smooth_coeff)

    dpeaks = detect_peaks(dnsmooth, mpd = 10*smooth_coeff,
                          edge='rising',show=False, mph=0.1)
    
    # Obtain the highest peak
    highest_dpeak = dpeaks[dnsmooth[dpeaks].argmax()]
    # Obtain the latest peak
    latest_dpeak = dpeaks[ydn[dpeaks].argmax()]
    # Getting the time difference between highest ones
    tdiff_dpeak = ydn[latest_dpeak] - ydn[highest_dpeak]

    if plot:
        fig = plt.figure(figsize=(8,8))
        ax1 = fig.add_subplot(211)
        jplot.unfilledBar(ybins, n)
        jplot.unfilledBar(ybins, nsmooth, color='red')
        plt.yscale('log')
        plt.plot((ybins[dpeaks]+ybins[dpeaks-1])/2., 
                 nsmooth[dpeaks], 'xg')
        
        used_peaks = np.array([highest_peak, latest_peak])
        plt.plot(ybins[used_peaks], nsmooth[used_peaks], 'ok')
        ax2 = fig.add_subplot(212,sharex=ax1)
        plt.plot(ydn, dn, color = 'blue')

        plt.plot(ydn, dnsmooth, color = 'red')
        plt.subplots_adjust(hspace=0.)
        plt.xlim(twindow)
        used_dpeaks = np.array([highest_dpeak, latest_dpeak])

        plt.plot((ydn[dpeaks]+ydn[dpeaks-1])/2., 
                  dnsmooth[dpeaks], 'xg')
        plt.plot(ydn[used_dpeaks], dnsmooth[used_dpeaks], 'ok')
        plt.show()
        #raw_input()
    return tdiff_peak, tdiff_dpeak

In [7]:
for one_infile in infile_list:
    infile_name = os.path.join(socdir, one_infile)
    
    
    
    print 'Run', one_infile
    reader = rat.socreader(infile_name)
    soc, run = reader.next()
    soc.GetRunID()
    
    outfile_name = os.path.join(peakdir, "%i" % soc.GetRunID()+ '.pckl')
    if os.path.isfile(outfile_name):
        print 'Run already processed. Skipping it ...'
        
        data = pickle.load(open(outfile_name))
        if not data.has_key('soc_pmts'):
            data['soc_pmts'] = np.array(soc.GetSOCPMTIDs())
            pickle.dump(data, open(outfile_name,'w'))
            print 'Adding SOC PMTs list'
        
        reader.close()
        continue
    # Open the file

    soc_pmts = np.array(soc.GetSOCPMTIDs())
    
    manip_pos = np.array(soc.calib.GetPos())
    fit_pos = np.array(soc.GetFitResult(soc.GetFitNames()[0]).GetVertex(0).GetPosition())

    
    data = {'manip_pos':manip_pos,
            'fit_pos':fit_pos,
            'soc_pmts':soc_pmts}
    

    zero_counter = 0
    skip_counter = 0
    
    delta_ts = np.zeros([npmts, 2])
    print 'Going over the ', len(soc_pmts), ' PMTs'

    for ipmt, one_pmt in enumerate(soc_pmts):
        time_array = np.array(soc.GetSOCPMT(one_pmt).GetTimes())
        if ipmt % 500 == 0:
            print ipmt,
        time_array[time_array < 0] = 0.
        if time_array.mean() == 0 or time_array.size < 1000 or np.sum(time_array>0) < 1000:
            skip_counter += 1
            continue

        delta_ts[one_pmt, :] = getTimeDiff(time_array,
                                            bin_width = bin_width)
        if np.sum(delta_ts[one_pmt,:]) == 0:
            zero_counter += 1

            
    data['delta_ts'] = delta_ts
    pickle.dump(data, open(outfile_name,'w'))
    print 'Zero counter', zero_counter
    print 'Skip counter', skip_counter
    
    reader.close()

Run SOC_0000017377_reprocessed.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000017375_reprocessed.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000017378_reprocessed.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000100558.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000100559.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000100560.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000100554.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000017386_reprocessed.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000100556.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000017376_reprocessed.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_0000100555.root
Run already processed. Skipping it ...
Adding SOC PMTs list
Run SOC_00000173